In [3]:
import os
import getpass
import glob
import ipywidgets as widgets
import scripts.filter_closing_smm as cfilter
import subprocess
from ipywidgets import VBox, HBox, Label, interactive, Box
from tqdm.notebook import trange, tqdm

user = getpass.getuser()
raw_path = f'/home/{user}/pysmm_downloads/0_raw/'

def return_paths(raw_path):
    """Create a list of folders in a given path
    skipping those with begin with '.' and are empty

    """
    paths = [folder for folder in os.listdir(raw_path) 
             if os.path.isdir(os.path.join(raw_path, folder)) and not folder.startswith('.') 
             and len(os.listdir(os.path.join(raw_path, folder))) != 0
            ]
    paths.sort()
    return paths

def return_images(path):
    tifs = glob.glob(path+'/*.tif')
    return tifs

def get_select2(*args):
    options = return_paths(os.path.join(raw_path, select_1.value))
    select_2.options = options

def closing_filter(process_path):
    IMAGES_TYPES = ('.tif')
    folder = process_path
    image_files = []    
    if os.path.isdir(folder):
        for root, dirs, files in os.walk(folder):
            if len(files) != 0:
                files = [os.path.join(root, x) for x in files if x.endswith(IMAGES_TYPES)]
                [image_files.append(os.path.abspath(file)) for file in files]
    else:
        print(f'ERROR: The {folder} joined is not a directory path.')
    print(f'There are {len(image_files)} images to process, please wait...')

    dates = []
    for i in trange(len(image_files)):
        dates.append(cfilter.extract_image_info(image_files[i]))
        cfilter.raw_to_processed(image_files[i])
    
    
def process(feature, field):
    print('Postprocessing the soil moisture maps')
    process_path = os.path.join(raw_path, feature, field)
    closing_filter(process_path)

def run():
    w = interactive(process, {'manual':True}, feature=select_1, field=select_2)
    w.children[0].description = ''
    w.children[1].description = ''
    w.children[2].description = 'Process the folder'
    
    display(HBox([Label('Select the folder you want process:'), w.children[0], w.children[1]]))
    display(HBox([w.children[2]]))
    display(w.children[-1])

raw_folders = []
    
select_1 = widgets.Select(
    options=return_paths(raw_path),
)

select_2 = widgets.Select()
select_1.observe(get_select2)    
    
run()

Output()

In [4]:
select_1.value

'103_phu_merged'

In [6]:
statistic = widgets.Dropdown(
    options=['median', 'mean', 'gmean', 'max', 'min', 'std', 'valid_pixels'],
    value='median',
    description='statistics:',
    disabled=False,
)

out_process_path = os.path.join(raw_path, select_1.value, select_2.value).replace('0_raw','1_processed')

def stack_composed(statistic):
    stat = statistic
    tifs = glob.glob(out_process_path+'/*.tif')
    process = subprocess.run(['stack-composed',
                                '-stat', stat,
                                '-bands', '1',
                                '-o', out_process_path]+
                                tifs,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE,
                            universal_newlines=True)
    
    if process.returncode == 0:
        print('Done!')
    else:
        print(process.stderr)
        


def run_stack():
    
    w = interactive(stack_composed, {'manual':True}, statistic=statistic)
    w.children[1].description = 'Run stack composed'
    display(w)

run_stack()

interactive(children=(Dropdown(description='statistics:', options=('median', 'mean', 'gmean', 'max', 'min', 's…